# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [110]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


zsh:1: command not found: jupyter


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [76]:
outputfile = "../output_data/cities.csv"
weather_data_df = pd.read_csv(outputfile)
weather_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Provideniya,64.38,-173.30,24.69,88,90,7.78,RU,1604100700
1,1,Rikitea,-23.12,-134.97,76.21,64,0,3.87,PF,1604100979
2,2,Souillac,-20.52,57.52,68.00,56,40,17.22,MU,1604100979
3,3,Puerto Ayora,-0.74,-90.35,70.00,93,66,4.00,EC,1604100979
4,4,Atuona,-9.80,-139.03,77.92,73,76,10.51,PF,1604100693
...,...,...,...,...,...,...,...,...,...,...
539,544,Padang,-0.95,100.35,79.07,75,93,3.22,ID,1604101430
540,545,Lima,-12.04,-77.03,64.40,77,90,11.41,PE,1604101309
541,546,Havelock,34.88,-76.90,60.80,59,20,10.29,US,1604101431
542,547,Podor,16.65,-14.96,88.90,13,75,10.58,MR,1604101431


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [104]:
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df['Humidity']
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

In [106]:
heat_layer.max_intensity = 100
heat_layer.point_radius = 2
heat_layer.gradient = ['white', 'red']

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [89]:
perfect_weather = weather_data_df.loc[(weather_data_df['Max Temp'] > 70) & (weather_data_df['Max Temp']  <= 80) & 
                                      (weather_data_df['Cloudiness'] <2) & 
                                      (weather_data_df['Wind Speed'] <10) ]

perfect_weather.drop(perfect_weather.columns[1], axis=1)
perfect_weather = perfect_weather.drop(perfect_weather.columns[0], axis=1)
perfect_weather = perfect_weather.reset_index()
perfect_weather = perfect_weather.dropna()
perfect_weather

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,1,Rikitea,-23.12,-134.97,76.21,64,0,3.87,PF,1604100979
1,67,Rayachoti,14.05,78.75,70.59,77,0,2.04,IN,1604101024
2,265,Bariloche,-41.15,-71.31,73.40,11,0,4.70,AR,1604101030
3,279,Saint George,37.10,-113.58,75.20,15,1,2.17,US,1604101208
4,356,Makkah al Mukarramah,21.43,39.83,75.36,63,0,2.64,SA,1604101295
5,360,Los Llanos de Aridane,28.66,-17.92,71.60,78,0,6.93,ES,1604101295
6,428,Umluj,25.02,37.27,77.50,38,0,5.08,SA,1604101341
7,468,Koulikoro,14.00,-7.75,73.00,26,0,2.68,ML,1604101384
8,525,Cape Canaveral,28.41,-80.60,71.60,68,1,5.82,US,1604101219


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [92]:
hotel_df = pd.DataFrame({'Hotel Name': "",
                        'City': perfect_weather['City'],
                        'Country': perfect_weather['Country'],
                         'Lat':perfect_weather['Lat'],
                         'Lng':perfect_weather['Lng']
                        }
                       )
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,,Rikitea,PF,-23.12,-134.97
1,,Rayachoti,IN,14.05,78.75
2,,Bariloche,AR,-41.15,-71.31
3,,Saint George,US,37.10,-113.58
4,,Makkah al Mukarramah,SA,21.43,39.83
5,,Los Llanos de Aridane,ES,28.66,-17.92
6,,Umluj,SA,25.02,37.27
7,,Koulikoro,ML,14.00,-7.75
8,,Cape Canaveral,US,28.41,-80.60


In [94]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotels",
    "keyword": "cheap",
    "key": g_key
}
# Use the lat/lng we recovered to identify cities with perfect weather
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

     # add location each iteration 
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "4 star" and our lat/lng ( I ran a search with lat amd lng the results were bad)
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#   print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}

In [95]:
hotel_df

,Hotel Name,City,Country,Lat,Lng
0,,Rikitea,PF,-23.12,-134.97
1,,Rayachoti,IN,14.05,78.75
2,,Bariloche,AR,-41.15,-71.31
3,,Saint George,US,37.10,-113.58
4,,Makkah al Mukarramah,SA,21.43,39.83
5,,Los Llanos de Aridane,ES,28.66,-17.92
6,,Umluj,SA,25.02,37.27
7,,Koulikoro,ML,14.00,-7.75
8,Discovery Beach Resort,Cape Canaveral,US,28.41,-80.60


In [107]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [109]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Rayachoti</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Bariloche</dd>\n<dt>Country</dt><dd>AR</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Saint George</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Makkah al Mukarramah</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Los Llanos de Aridane</dd>\n<dt>Country</dt><dd>ES</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Umluj</dd>\n<dt>Country</dt><dd>SA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd></dd>\n<dt>City</dt><dd>Koulikoro</dd>\n<dt>Country</dt><dd>ML</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Discovery Beach Resort</dd>\n<dt>City</dt><dd>Cape Canaveral</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n']

In [108]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,hotel_info)
# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…